In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
#create a Model class that is inherits neural network nn.Module
class Model(nn.Module):
    #input layer with 4 features of flower from our datasets --> 
    #choose to hidden Layer1(H1)(number of neurons in it)--> H2(n)--> 
    #output (3classes of flower)
    def __init__(self, in_features=4, h1=8,h2=9,OutputLayers=3):
        super().__init__() #instantiate our nn.module
        #fully connect 
        self.fc1=nn.Linear(in_features,h1)
        self.fc2=nn.Linear(h1,h2)
        self.out=nn.Linear(h2,OutputLayers)
        
    def forward(self,x):
        #rectified linear unit
        #start to layer one then move to layer 2
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        
        return x
        
        

SyntaxError: expected ':' (3873504301.py, line 13)

In [ ]:
#pick a manual seed for randomization
torch.manual_seed(32)
# Create an instance of model
model = Model()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
url='https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
#data frame
my_df=pd.read_csv(url)

In [ ]:
my_df.tail()

In [ ]:
my_df.head()

In [ ]:
#change the last column from string to int
#my_df['variety']=my_df['variety'].replace('Setos',0.0)
#my_df['variety']=my_df['variety'].replace('Versicolor',1.0)
#my_df['variety']=my_df['variety'].replace('Virginica',2.0)

my_df['variety'] = my_df['variety'].replace({'Setosa': 0, 'Versicolor': 1, 'Virginica': 2})
# Handle future downcasting
my_df = my_df.infer_objects(copy=False)
# Split features and labels
X = my_df.drop('variety', axis=1)
#train test and split set X,Y
my_df

In [ ]:
#Convert its datatype to int64 (64-bit integer)
y = my_df['variety'].astype(np.int64)


In [ ]:
#convert these to numpy arrays
X= X.values
y= y.values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:

#train test split testsize 20%, Train size is 80%
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=32)

In [ ]:
#convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)


In [ ]:
#convert Y features to Long tensors
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
#set the criterion of model to measure the error, how far off the predictions are from the data
criterion=nn.CrossEntropyLoss()
#choose Adam Optimizer, lr = learning rate (if error doesn't go down after a bunch of iterations (epochs), lower learning rate)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
# Train our model
# Epochs?(one run thru all the training data in our network), current we set spin 100 time
epochs = 100
losses = []
for i in range(epochs):
    # Go forward and get a prediction
    y_pred = model.forward(X_train) #get predicted results

    # We want to measure the loss/error, it will be high at first
    loss = criterion(y_pred, y_train) #predicted values vs the y_train

    #keep Track of our losses
    losses.append(loss.detach().numpy())
    #print every 10 epochs
    if i%10==0:
        print(f'epochs:{i} and loss: {loss}')

    #do some back propogation by take the error rate of forward propagation and feed it back
    #thru the network to fine tune the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
#graph the plot
plt.plot(range(epochs), losses)
plt.ylabel("Validation Loss/error")
plt.xlabel("Epoch")

In [ ]:
# Evaluate Model on Test Data Set (Validate model on test set)
with torch.no_grad(): #turn off back propogation
    y_eval = model.forward(X_test) # X_test are features from out test set, y_eval wil be predictions
    loss = criterion(y_eval,y_test) # Find the loss or error

In [ ]:
loss


In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val= model.forward(data)

        if y_test[i]==0:
            x="Setsosa"
        elif y_test[i]==1:
            x="Versicolor"
        else:
            x="Virginica"
            
        print(f'{i+1}.) {str(y_val)} \t{x} \t {y_val.argmax().item()}')

        #let find out if this result is correct or not
        if y_val.argmax().item() ==y_test[i]:
            correct +=1
print(f'We got the result {correct} predicted correct!')

In [ ]:
#feed new data
new_iris= torch.tensor([4.7, 3.2, 1.3, 0.2])

In [ ]:
with torch.no_grad():
    print(model(new_iris))